julia EM model fitting example, Nathaniel Daw 8/2020

####### NOTE NOTE NOTE: PARALLEL COMPUTATION IS NOW AUTOMATIC IN THIS VERSION 
####### BUT TO RUN PARALLEL YOU MUST SET ENVIRONMENT VARIABLE JULIA_NUM_THREADS  
####### BEFORE STARTING JULIA OR JUPYTER-NOTEBOOK

eg in linux/bash:
      export JULIA_NUM_THREADS=`nproc`; julia

to install dependencies run:
import Pkg
Pkg.add(["DataFrames", "ForwardDiff", "Optim", "LinearAlgebra", "StatsFuns", "SpecialFunctions",  "Statistics", "Distributions", "GLM"])


In [1]:
###### setup 

full = false    # Maintain full covariance matrix (vs a diagional one) at the group level
emtol = 1e-3    # stopping condition (relative change) for EM

# load the code
# change this to where you keep the code
# directory = "/mnt/c/Users/daw/Dropbox (Princeton)/expts/julia em/git/em"
directory = "/users/ndaw/Dropbox (Princeton)/expts/julia em/git/em"

push!(LOAD_PATH,directory)
using EM

# this loads additional packages used in examples below

using Statistics
using Random
using GLM
using DataFrames


In [2]:
###### Q learning example

# simulate some  qlearning data

Random.seed!(1234); # (for repeatability)

NS = 250;
NT = 200;
NP = 2;

params = zeros(NS,NP);

cov = randn(NS); # simulated between-subject variable, e.g. age or IQ
cov = cov .- mean(cov);

cov2 = randn(NS); # simulated between-subject variable, e.g. age or IQ
cov2 = cov2 .- mean(cov2);

# subject level parameters

params[:,1] = 1 .+ 0.5 * randn(NS) + cov; # softmax  temp: mean 1, effect of cov
params[:,2] = 0 .+ 1 * randn(NS) + cov2;  # learning rate (pre sigmoidal transform): mean 0, effect of cov2

c = zeros(Int64,NS*NT);
r = zeros(Int64,NS*NT);
s = zeros(Int64,NS*NT);

for i = 1:NS
	(c[(i-1)*NT+1:i*NT],r[(i-1)*NT+1:i*NT]) = simq(params[i,:],NT);
	s[(i-1)*NT+1:i*NT] .= i;
end

data = DataFrame(sub=s,c=c,r=r);
subs = 1:NS;

In [3]:
# set up the fit

# design matrix specifying the group level model
# this is replicated once for each model parameter
#
# in particular for each subject-level parameter x_ij  (subject i, parameter j)
#
# x_ij ~ Normal(X beta_j, Sigma)
#
# thus X has a row for each subject and a column for each predictor
# in the simplest case where the only predictor is an intercept, X = ones(NS)
# then beta_j specifies the group-level mean for parameter j
#
# but in this example we have two covariates that vary by subject
# so x_ij = beta_1j + beta_2j * cov_i + beta_3j * cov2_i
# and we infer the slopes beta for each parameter j as well as the intercept
#
# so we have a design matrix with 3 columns, and a row per subject:

X = [ones(NS) cov cov2];

# note: when you have no covariates (only intercepts) omit the brackets to get a column vector

# X = ones(NS)

# starting points for group level parameters
# betas: one column for each parameter, one row for each regressor (so here: 3 rows, 2 columns)
# make sure these are floats
# note: if you have a single predictor you need a row vector (length: # params)
# eg betas = [0. 0.];
# and if there is also only a single model parameter and no covariates, then betas is a scalar
# eg betas = 0.

startbetas = [1. 0; 0 0; 0 0]

# sigma: one element starting variance for each model parameter (this is really variance not SD)
# if there is only one model parameter it needs to be a length-one vector eg. sigma = [5.]

startsigma = [5., 1]



2-element Vector{Float64}:
 5.0
 1.0

Estimation & significance tests

In [4]:

# fit the model
# (this takes: a data frame, a list of subjects, a group level design matrix, 
#  starting group level betas, starting group-level variance or covariance, a likelihood function
#  and some optional options)
#
# (return values: betas are the group level means and slopes
#  sigma is the group level *variance* or covariance
#  x is a matrix of MAP/empirical Bayes per-subject parameters
#  l is the per-subject negative log likelihoods 
#  h is the *inverse* per subject hessians) 

(betas,sigma,x,l,h) = em(data,subs,X,startbetas,startsigma,qlik; emtol=emtol, full=full);

betas # ground truth would be [1 0 ; 1 0; 0 1] so this is closei


iter: 9
betas: [0.92 -0.23; 0.99 0.1; -0.04 0.76]
sigma: [0.27, 0.74]
free energy: -22995.277507
change: [0.00021, -0.000331, 0.000284, 9.2e-5, -0.000974, 3.9e-5, 0.000672, 0.000147]
max: 0.000974


3×2 Matrix{Float64}:
  0.915928   -0.233291
  0.985689    0.104455
 -0.0409915   0.756387

In [5]:
# standard errors on the subject-level means, based on an asymptotic Gaussian approx 
# (these may be inflated for small n)
# returns standard errors, pvalues, and a covariance matrix 
# these are a vector ordered as though the betas matrix were read out column-wise
# eg parameter 1, (intercept covariate covariate) then parameter 2

(standarderrors,pvalues,covmtx) = emerrors(data,subs,x,X,h,betas,sigma,qlik)
reshape(pvalues,size(betas'))'  # cov1 (2nd row) is significant for beta (first column)
# & cov2 (3rd row) is significant for alpha (second column)

3×2 adjoint(::Matrix{Float64}) with eltype Float64:
 3.13697e-88  0.00011836
 1.79943e-84  0.0712136
 0.291688     1.26102e-26

In [6]:
# another way to get a p value for a covariate, by omitting it from the model and regressing
# this seems to work better when full=false
# in general not super well justified and can clearly be biased in some cases
# but works well in practice as long as you avoid the bias cases (which are pretty obvious)

X2 = ones(NS);
startbetas2 = [0. 0.];
startsigma2 = [5., 1];
(betas2,sigma2,x2,l2,h2) = em(data,subs,X2,startbetas2,startsigma2,qlik; emtol=1e-5, full=full);

display(lm(@formula(beta~cov+cov2),DataFrame(beta=x2[:,1],cov=cov,cov2=cov2)))
display(lm(@formula(alpha~cov+cov2),DataFrame(alpha=x2[:,2],cov=cov,cov2=cov2)))

# again the first covariate is significant for beta and the second for alpha

StatsModels.TableRegressionModel{LinearModel{GLM.LmResp{Vector{Float64}}, GLM.DensePredChol{Float64, LinearAlgebra.CholeskyPivoted{Float64, Matrix{Float64}}}}, Matrix{Float64}}

beta ~ 1 + cov + cov2

Coefficients:
──────────────────────────────────────────────────────────────────────────
                  Coef.  Std. Error      t  Pr(>|t|)  Lower 95%  Upper 95%
──────────────────────────────────────────────────────────────────────────
(Intercept)   0.892822    0.0350906  25.44    <1e-70   0.823707   0.961936
cov           0.882962    0.0347236  25.43    <1e-70   0.814569   0.951354
cov2         -0.0350505   0.0378879  -0.93    0.3558  -0.109675   0.039574
──────────────────────────────────────────────────────────────────────────

StatsModels.TableRegressionModel{LinearModel{GLM.LmResp{Vector{Float64}}, GLM.DensePredChol{Float64, LinearAlgebra.CholeskyPivoted{Float64, Matrix{Float64}}}}, Matrix{Float64}}

alpha ~ 1 + cov + cov2

Coefficients:
───────────────────────────────────────────────────────────────────────────
                  Coef.  Std. Error      t  Pr(>|t|)   Lower 95%  Upper 95%
───────────────────────────────────────────────────────────────────────────
(Intercept)  -0.200008    0.0501431  -3.99    <1e-04  -0.29877    -0.101245
cov           0.0821765   0.0496187   1.66    0.0990  -0.0155532   0.179906
cov2          0.622858    0.0541403  11.50    <1e-24   0.516222    0.729493
───────────────────────────────────────────────────────────────────────────


iter: 9
betas: [0.89 -0.2]
sigma: [1.17, 1.09]
free energy: NaN
change: [2.0e-6, -8.0e-6, 7.0e-6, 6.0e-6]
max: 8.0e-6


Model comparison metrics

In [7]:
# laplace approximation to the aggregate log marginal likelihood of the whole dataset
# marginalized over the individual params

ll1 = lml(x,l,h)

22371.741108231596

In [8]:
# to compare these between models you need to correct for the group level free parameters
# either aic or bic (this is Quentin Huys' IBIC or IAIC, i.e. the subject level
# params are marginalized by laplace approx, and aggregated, and the group level
# params are corrected by AIC or BIC)

ibic(x,l,h,betas,sigma,NS*NT)


22415.020221369236

In [9]:
iaic(x,l,h,betas,sigma)

22379.741108231596

In [10]:
# or by computing unbiased per subject marginal likelihoods via cross validation.
# you can do paired t tests on these between models
# these are also appropriate for SPM_BMS etc

liks = loocv(data,subs,x,X,betas,sigma,qlik;emtol=emtol, full=full)
sum(liks)

# note that iaic does an excellent job of predicting the aggregate held out likelihood
# but importantly these are per subject scores that you can compare in paired tests
# across models as per Stephan et al. random effects model comparison

Subject: 1..2..3..4..5..6..7..8..9..10..11..12..13..14..15..16..17..18..19..20..21..22..23..24..25..26..27..28..29..30..31..32..33..34..35..36..37..38..39..40..41..42..43..44..45..46..47..48..49..50..51..52..53..54..55..56..57..58..59..60..61..62..63..64..65..66..67..68..69..70..71..72..73..74..75..76..77..78..79..80..81..82..83..84..85..86..87..88..89..90..91..92..93..94..95..96..97..98..99..100..101..102..103..104..105..106..107..108..109..110..111..112..113..114..115..116..117..118..119..120..121..122..123..124..125..126..127..128..129..130..131..132..133..134..135..136..137..138..139..140..141..142..143..144..145..146..147..148..149..150..151..152..153..154..155..156..157..158..159..160..161..162..163..164..165..166..167..168..169..170..171..172..173..174..175..176..177..178..179..180..181..182..183..184..185..186..187..188..189..190..191..192..193..194..195..196..197..198..199..200..201..202..203..204..205..206..207..208..209..210..211..212..213..214..215..216..217..218..219..220.

22379.640703784487